In [0]:
#!/usr/bin/env python
import tensorflow as tf
import numpy as np
import h5py
import time
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from skimage.transform import radon, iradon
from skimage.measure import compare_ssim
import math
from google.colab import drive    # in ordre to mount drive to colab
drive.mount('/content/gdrive')

gpu_options = tf.GPUOptions(allow_growth=True)
session_config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options, allow_soft_placement=True)
sess = tf.Session(config=session_config)

batch_size = 10
learning_rate = 1.0e-4
width = 64       # dimension of k-space data and image data are the same
lambda_sl = 0.5
use_gan = True # change to False if not using GAN (LS_GAN).
if use_gan:
  disc_iters = 4
  lambda_al = 0.0025
else:
  disc_iters = 0
  lambda_al = 0

num_epoch = 10

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def read_data(file_name, if_complex=False):
    if if_complex:
      f = h5py.File(file_name, 'r')
      label = np.array(f['data'], dtype=np.complex64)
      f.close()
      label = np.append(np.expand_dims(np.real(label),-1), np.expand_dims(np.imag(label), -1), -1)
      print(label.shape)
    else:
      f = h5py.File(file_name, 'r')
      label = np.array(f['data'], dtype=np.float32)
      f.close()
    return label

In [0]:
    def generator(x):
        x = tf.reshape(x, [-1, 64 * 64 * 2])  # input contains both real and complex parts.
        output = tf.contrib.layers.fully_connected(x,
                                               64 * 64 * 2,
                                               activation_fn=tf.tanh,
                                               weights_initializer=tf.contrib.layers.xavier_initializer(),
                                               biases_initializer=tf.zeros_initializer(),
                                               trainable=True)       # first FC layer with 64 * 64 * 2 neurons
        print(output)
        output = tf.layers.batch_normalization(output)     # batch normalization
                
        output = tf.contrib.layers.fully_connected(output,
                                               64 * 64,
                                               activation_fn=tf.tanh,
                                               weights_initializer=tf.contrib.layers.xavier_initializer(),
                                               weights_regularizer=None,
                                               biases_initializer=tf.zeros_initializer(),
                                               trainable=True,)# second FC layer with 64 * 64 neurons
    
        output = tf.layers.batch_normalization(output) # batch normalization
            
        output = tf.reshape(output, [-1, 64, 64, 1]) # get images
        
        output = tf.layers.conv2d(output, 
                                  filters=64, 
                                  kernel_size=3,
                                  strides=1,
                                  padding='SAME',
                                  activation=tf.nn.relu,
                                  use_bias=True,
                                  bias_initializer=tf.zeros_initializer(),
                                  trainable=True)   # convolutional layers used to denosing images
    
        output = tf.layers.batch_normalization(output)
    
        output = tf.layers.conv2d(output, 
                                  filters=64, 
                                  kernel_size=3,
                                  strides=1,
                                  padding='SAME',
                                  activation=tf.nn.relu,
                                  use_bias=True,
                                  bias_initializer=tf.zeros_initializer(),
                                  trainable=True)
        
        output = tf.layers.batch_normalization(output)
        
        output = tf.layers.conv2d(output, 
                                  filters=1, 
                                  kernel_size=3,
                                  strides=1,
                                  padding='SAME',
                                  activation=tf.nn.relu,
                                  use_bias=True,
                                  bias_initializer=tf.zeros_initializer(),
                                  trainable=True)
        
        return output

In [0]:
def discriminator(x):
    reuse=tf.AUTO_REUSE   
    outputs = tf.layers.conv2d(x, 64, 3, padding='SAME', kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv1')
    outputs = tf.nn.relu(outputs)
        
    outputs = tf.layers.conv2d(outputs, 64, 3, padding='SAME', strides=(2,2), kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv2')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 128, 3, padding='SAME', kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv3')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 128, 3, padding='SAME', strides=(2,2), kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv4')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 256, 3, padding='SAME', kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv5')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 256, 3, padding='SAME', strides=(2,2), kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv6')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.dense(outputs, units=1024, name='dense1')
    outputs = tf.nn.relu(outputs)
    outputs = tf.layers.dense(outputs, units=1, name='dense2')
    return outputs

In [0]:
lr = tf.placeholder(dtype=tf.float32, shape=[])  # learning rate
real_placeholder = tf.placeholder(dtype=tf.float32, shape=[batch_size, width, width, 1], name='real')  # placeholder for images
fake_placeholder = tf.placeholder(dtype=tf.float32, shape=[batch_size, width, width, 2], name='fake')  # placeholder for k-space data (complex). one channel contains real part
                                                                                                       # and the other channel contains the complex part
with tf.variable_scope('generator_model', reuse=tf.AUTO_REUSE) as scope_generator_model:
    Gz = generator(fake_placeholder)                                 # feed k-space data into the network
print(Gz)
with tf.variable_scope('discriminator_model', reuse=tf.AUTO_REUSE) as scope_discriminator_model:
    FD_d = discriminator(real_placeholder)                   # same discriminator network as in the autoencoder
    LD_d = discriminator(Gz)
print(FD_d)
print(LD_d)


disc_loss = tf.reduce_mean((FD_d - 1)**2 + (LD_d - 0)**2)
disc_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='discriminator_model')
d_trainer = tf.train.AdamOptimizer(learning_rate=lr).minimize(disc_loss, var_list=disc_variables)
# training discriminator

ssim = tf.squeeze(tf.reduce_mean(tf.image.ssim(Gz, real_placeholder, 1.0)))
ssim_loss = 1 - ssim
mse = tf.reduce_mean(tf.squared_difference(Gz, real_placeholder))
gen_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='generator_model')
gen_cost = tf.reduce_mean((LD_d - 1)**2)

g_trainer = tf.train.AdamOptimizer(learning_rate=lr).minimize(mse + lambda_al * gen_cost + lambda_sl * ssim_loss, var_list=gen_variables)
# training generator

saver = tf.train.Saver()
print("enter Session")
sess.run(tf.global_variables_initializer())  # initializing parameters
#saver.restore(sess, '/content/gdrive/My Drive/hands_on4/Automap/model_dm/SIGmathmodel100.ckpt')   # used to restore medels
print('model restored')
print("Begin training")
label = read_data('/content/gdrive/My Drive/hands_on4/Automap/Train/Label.hdf5')   # read label
data = read_data('/content/gdrive/My Drive/hands_on4/Automap/Train/Data.hdf5', if_complex=True)  # read data


for iteration in range(num_epoch):
    #break
    label, data = shuffle(label, data)
    val_lr = learning_rate / 2
    num_batches = data.shape[0] // batch_size
    for i in range(num_batches):
        for _ in range(disc_iters):
            idx = np.random.permutation(label.shape[0])   # input to the discriminator and the input to the generator should be different
            batch_label = label[idx[:batch_size]]
            batch_label = np.expand_dims(batch_label, -1)
            
            batch_data = data[idx[:batch_size]]
            
            _ = sess.run(d_trainer, feed_dict={real_placeholder: batch_label,
                                               fake_placeholder: batch_data,
                                               lr: val_lr})  # training discriminator
            
        batch_label = label[i * batch_size: (i + 1) * batch_size] # input to the discriminator and the input to the generator should be different
        batch_label = np.expand_dims(batch_label, -1)
        batch_data = data[i * batch_size: (i + 1) * batch_size]
        _mse, _ssim, _ = sess.run([mse, ssim, g_trainer], feed_dict={real_placeholder: (batch_label),
                                                                             fake_placeholder: (batch_data), lr: val_lr})
        # training generator

        print('Epoch: %d - %d - mse %.6f: - ssim %.6f: ' % (iteration, i, _mse, _ssim))

saver.save(sess, '/content/gdrive/My Drive/hands_on4/Automap/model_dm/SIGmathmodel100.ckpt')
  
print("testing dataset")
test_label = read_data('/content/gdrive/My Drive/hands_on4/Automap/Train/Label.hdf5')      # read label  
test_data = read_data('/content/gdrive/My Drive/hands_on4/Automap/Train/Data.hdf5', if_complex=True)  # read data
for c in range(test_label.shape[0] // batch_size):
    test_batch_label = test_label[c * batch_size: (c + 1) * batch_size]
    test_batch_label = np.expand_dims(test_batch_label, -1)
    
    test_batch_data = test_data[c * batch_size: (c + 1) * batch_size]
    
    with tf.variable_scope('generator_model') as scope:
        scope.reuse_variables()
        estimated = sess.run(Gz, feed_dict={fake_placeholder: test_batch_data})
    for i in range(batch_size):
      t_r = np.squeeze(test_batch_label[i])
      e_1 = np.squeeze(estimated[i])

      f_n = '/content/gdrive/My Drive/hands_on4/Automap/testing_results/' + str(c) + str(i) + 'validation.jpg'  # save reconstructed images
      plt.imsave(f_n, e_1, cmap='gray')
      f_n = '/content/gdrive/My Drive/hands_on4/Automap/testing_results/' + str(c) + str(i) + 'real_.jpg'  # save real images
      plt.imsave(f_n, t_r, cmap='gray')

      z = (t_r - e_1) ** 2
      print(np.mean(z))   # print mse loss

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("generator_model/fully_connected/Tanh:0", shape=(10, 8192), dtype=float32)
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Tensor("generator_model/conv2d_2/Relu:0", shape=(10, 64, 64, 1), dtype=float32)
Instructions for updating:
Use keras.layers.Dense instead.
Tensor("discriminator_model/dense2/BiasAdd:0", shape=(

ValueError: ignored